# Import

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [4]:
# DataFrame
URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
# Column names of the DataFrame
column_names = ['MPG', 'Cylinders', 'Displacement',
                'Horsepower', 'Weight', 'Acceleration', 
                'Model Year', 'Origin']

# Importing DataFrame
dataset = pd.read_csv(URL, names=column_names, na_values='?',
                    comment='\t', sep=' ', skipinitialspace=True)

In [ ]:
dataset.tail()